Import

In [1]:
from classification.datasets import Dataset
from classification.utils.audio_student import Feature_vector_DS
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from classification.utils.utils import accuracy
import numpy as np
import os, threading
import matplotlib.pyplot as plt
from time import time
import pickle as pickle

Initialisation

In [2]:
### TO RUN
dataset = Dataset()
classnames = dataset.list_classes()
### TO RUN
PATH = "data/feature_freq/"  # where to save the features matrices
model_dir = "data/models/"  # where to save the models
os.makedirs(PATH, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [3]:
myds = Feature_vector_DS(dataset, Nft=512, nmel=20, duration=950, shift_pct=0.0, fs=11111)
train_pct = 0.7
featveclen = len(myds["fire", 0])  # number of items in a feature vector
nitems = len(myds)  # number of sounds in the dataset
naudio = dataset.naudio  # number of audio files in each class
nclass = dataset.nclass  # number of classes
nlearn = round(naudio * train_pct)  # number of sounds among naudio for training

data_aug_factor = 1
class_ids_aug = np.repeat(classnames, naudio * data_aug_factor)

"Compute the matrixed dataset, this takes some seconds, but you can then reload it by commenting this loop and decommenting the np.load below"
X = np.zeros((data_aug_factor * nclass * naudio, featveclen))
for s in range(data_aug_factor):
    for class_idx, classname in enumerate(classnames):
        for idx in range(naudio):
            featvec = myds[classname, idx]
            X[s * nclass * naudio + class_idx * naudio + idx, :] = featvec
np.save(PATH + "feature_matrix_2D.npy", X)

class_ids_aug = np.repeat(classnames, naudio * data_aug_factor)
y = class_ids_aug.copy()

fonction pour thread

In [4]:

model_knn = RandomForestClassifier(150)

t1 = time()
pca = PCA(n_components=8,whiten=True)
myds_th = Feature_vector_DS(dataset, Nft=512, nmel=20, duration=5000, shift_pct=0.0,fs=11111)
featveclen=len(myds_th['fire', 0])
X = np.zeros((data_aug_factor * nclass * naudio, featveclen))
for s in range(data_aug_factor):
    for class_idx, classname in enumerate(classnames):
        for idx in range(naudio):
            featvec = myds_th[classname, idx]
            X[s * nclass * naudio + class_idx * naudio + idx, :] = featvec
np.save(PATH +str(111111)+ "feature_matrix_2D.npy", X)

#classification
acc = []
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)  # random_state=1
new_train, new_test = np.zeros((len(X_train),400)),np.zeros((len(X_test),400))
for i in range(len(X_train)):
    new_train[i] = X_train[i][:400]

for i in range(len(X_test)):
    new_test[i] = X_test[i][:400]

X_train = new_train
X_test = new_test

mean = np.mean(X_train, axis=0)
X_train_mean=X_train-mean
X_learn_normalised = X_train_mean/ np.linalg.norm(X_train_mean, axis=1, keepdims=True)

mean = np.mean(X_test, axis=0)
X_val_mean=X_test-mean
X_val_normalised = X_val_mean/ np.linalg.norm(X_val_mean, axis=1, keepdims=True)

    
X_learn_normalised = pca.fit_transform(X_learn_normalised)
X_val_normalised = pca.transform(X_val_normalised)
pickle.dump(pca, open("./pca_vite", 'wb'))

if(len(X_learn_normalised[0])!=len(X_val_normalised[0])):
    raise(f"ERrOR {len(X_learn_normalised[0])}, {len(X_val_normalised[0])}")

model_knn.fit(X_learn_normalised, y_train)
prediction_knn = model_knn.predict(X_val_normalised)
print(accuracy(prediction_knn, y_test))
pickle.dump(model_knn, open("./model_vite", 'wb'))


0.7066666666666667
